In [1]:
from scipy.special import binom as binomial
#compute probabilities of for given branch in the binary tree
def split_weights(n,k):
    v0=binomial(n-1,k)
    v1=binomial(n-1,k-1)
    return v0/(v0+v1),v1/(v0+v1)

#compute all probabilities for the binary tree
def circuit_layout(n,k):
    L=[]
    for i in range(n):
        for j in range(min(i,k-1),-1,-1):
            if n-i >= k-j:
                L.append([n-i,k-j,split_weights(n-i,k-j)])
    return L

In [4]:
from qibo.models import Circuit
from qibo import gates
import numpy as np
def create_qibo_circuit(n,k):
    c=Circuit(int(n+k+(n+1)*k))
    L=circuit_layout(n,k)
    #print(L)
    cur=n
    c.add(gates.X(n+k-1))

    for i in L:
        if(i[2]==(0,1)):
            c.add(gates.CNOT(n+i[1]-1,n-i[0]))
        else:
            if i[0]!=n:
                c.add(gates.CRY(n+i[1]-1,n-i[0],float(2*np.arccos(np.sqrt(i[2][0])))))
            else:
                c.add(gates.RY(0,float(2*np.arccos(np.sqrt(i[2][0])))))
        if i[1]!=1:
            g=gates.SWAP(n-1+i[1],n-1+i[1]-1)
            g.controlled_by(n-i[0])
            c.add(g)
        else:
            c.add(gates.CNOT(n-i[0],n))
    return c

In [10]:
#example for 5-bit values with 3 bits set (n=5, k=3)
n=5
k=3
c=create_qibo_circuit(5,3)
c.add(gates.M(*[0,1,2,3,4]))
measure=c(nshots=int(10000))
for i in measure.frequencies():
    print(i,measure.frequencies()[i]/10000)

00111 0.1016
01011 0.1077
01101 0.0975
01110 0.1007
10011 0.0985
10101 0.0986
10110 0.0986
11001 0.0933
11010 0.1021
11100 0.1014


In [22]:

n=3
k=3
perm=[i for i in range(n)]
ancillas=[i for i in range(n,n+k)]

H=np.arange((n+1)*k).reshape(k,n+1)
c=n+k

for j in range(n+1):
    for i in range(k):
        H[i,j]=c
        c+=1
H

array([[ 6,  9, 12, 15],
       [ 7, 10, 13, 16],
       [ 8, 11, 14, 17]])

# Define all the Helper Functions

In [11]:
def set_ancillas_to_num(c,ancillas,num):
    ind=0
    for i in reversed(bin(num)[2:]):
        if int(i)==1:
            c.add(gates.X(ancillas[ind]))
        ind+=1
    

In [12]:
def add_negates_for_check(c,ancillas,num):
    ind=0
    for i in reversed(bin(num)[2:]):
        if int(i)==0:
            c.add(gates.X(ancillas[ind]))
        ind+=1
    for i in range(len(bin(num)[2:]),len(ancillas)):
        c.add(gates.X(ancillas[i]))
    

In [13]:
def controlls_for_check_num(ancillas,num):
    l=[]
    c=0
    for i in reversed(bin(num)[2:]):
        if int(i)==1:
            l.append(ancillas[c])
        c+=1
    return l

In [14]:
def sub_one(c,ancillas,controlls):
    a=ancillas
    c.add(gates.X(a[0]).controlled_by(*controlls))
    for i in range(1,len(a)):
        controlls.append(a[i-1])
        c.add(gates.X(a[i]).controlled_by(*controlls))
        

In [47]:
def add_one(c,ancillas,controlls):
    a=ancillas
    for i in range(len(a)-1):
        controlls.append(a[i])

    #c.add(gates.X(a[0]).controlled_by(*controlls))
    for i in reversed(range(0,len(a))):
        c.add(gates.X(a[i]).controlled_by(*controlls))
        controlls.pop()

In [16]:
def row_add(c,col,row_res,row_addend,H,n,controlls):
    for i in range(col+1,n+1):
        g=gates.X(H[row_res,i])
        controlls.append(H[row_addend,i])
        c.add(g.controlled_by(*controlls))
        controlls.pop()
        
    

In [17]:
def add_pivot(c,identity,parity,H,n,r,controlls):
    i=identity
    p=parity
    c.add(gates.X(H[i,p]).controlled_by(*controlls))
    #loop for differnt rows in which the pivot is searched
    for l in range(r-(i+1)):
        #current row in which pivot is searched
        pr=i+l+1
        #need to add controls on permutation
        row_add(c,p,i,pr,H,n,[H[i,p]]+[H[i+1+o,p] for o in range(l+1)]+controlls)
        #add not for "if all preious were 0"-statement
        if l!=r-i-2:
            c.add(gates.X(H[pr,p]).controlled_by(*controlls))
    #revert all nots
    for l in range(r-i-1):
        c.add(gates.X(H[i+l,p]).controlled_by(*controlls))
    

In [18]:
def lower_col_elimination(c,identity,parity,H,n,r,controlls):
    i=identity
    p=parity
    #for each element below the current diagonal
    for l in range(r-i-1):
        #add row if needed to eliminate ones in column p
        row_add(c,p,i+l+1,i,H,n,[H[i+l+1,p]]+controlls)

In [19]:
def upper_col_elimination(c,identity,parity,H,n,r,controlls):
    i=identity
    p=parity
    #for each row above the current diagonal entry (identiy)
    for l in range(i):
        #eliminate ones by adding row[i] to that row
        c.add(gates.X(H[l,n]).controlled_by(*([H[l,p],H[i,n]]+controlls)))

# Solving a linear system of equations

In [20]:
n=3
r=3

H=np.arange((n+1)*r).reshape(r,n+1)
c=0

for j in range(n+1):
    for i in range(r):
        H[i,j]=c
        c+=1


In [21]:

while True:
    A=np.random.randint(2, size=(r, n))
    if np.linalg.matrix_rank(A)==r:
        break

b=np.random.randint(2, size=(r, 1))
print("solve A*x=b for x, where" )
print("")
print("A =")
print(A)
print("")
print("b =")
print(b)
print("")
print("x =")
print(np.linalg.solve(A,b)%2)

solve A*x=b for x, where

A =
[[0 1 0]
 [1 0 0]
 [0 0 1]]

b =
[[1]
 [0]
 [1]]

x =
[[0.]
 [1.]
 [1.]]


In [22]:

c=Circuit((n+1)*r)

#initialize matrix in quantum register
for i in range(r):
    for j in range(n):
        if A[i, j] == 1:
            c.add(gates.X(H[i, j]))
for i in range(r):
    if b[i] == 1:
        c.add(gates.X(H[i, n]))
        
#add pivots and create upper triangular matrix
for i in range(r-1):
    add_pivot(c,i,i,H,n,r,[])
    lower_col_elimination(c,i,i,H,n,r,[])

#solve via back substitution
for i in reversed(range(1,r)):
    #upper_col_elimination(c,0,0,H,n,r)
    upper_col_elimination(c,i,i,H,n,r,[])


In [23]:

c.add(gates.M(*[H[i,n] for i in range(r)]))
print(c.draw())

q0 : ─X─o─o─o───o─o─o─X─────────────────────────────────
q1 : ─X─o─o─o─X─o─o─o─X─o─o─o───────────────────────────
q2 : ───|─|─|───o─o─o───|─|─|─o─o─o─────────────────────
q3 : ─X─X─|─|───X─|─|───o─|─|─o─|─|─────────────────o───
q4 : ───o─|─|───|─|─|───X─|─|─|─|─|─X─o─o─X─────────|───
q5 : ─────|─|───o─|─|─────|─|─X─|─|───o─o───o─o─────|───
q6 : ─────X─|─────X─|─────o─|───o─|───|─|───|─|─o───|───
q7 : ─────o─|─────|─|─────X─|───|─|───X─|───o─|─|─o─|───
q8 : ─X─────|─────o─|───────|───X─|───o─|───X─|─|─|─|───
q9 : ─X─────X───────X───────o─────o─────|─────|─X─|─X─M─
q10: ───────o───────|───────X─────|─────X─────o─|─X─o─M─
q11: ─X─────────────o─────────────X─────o─────X─o─o───M─


In [24]:
measure=c(nshots=1)

In [25]:
measure.frequencies()

Counter({'011': 1})

# Combining both Circuits

In [105]:
import itertools
r=2
n=4
solutions=[]

while(len(solutions)<2):
    solutions=[]
    A=np.random.randint(2, size=(r, n))
    b=np.random.randint(2, size=(r,1))
    comb=itertools.combinations([i for i in range(n)],r)
    Aprime=np.arange(r*r).reshape(r,r)
    for i in comb:
        ind=0
        for j in i:
            for l in range(r):
                Aprime[l,ind]=A[l,j]
            ind+=1
        mask=0
        for j in i:
            mask^=(1<<(n-j-1))
        v=bin(mask)[2:]
        while (len(v)<n):
            v="0"+v
        try:
            sol=np.linalg.solve(Aprime,b)%2
            for it in reversed(sol):
                if it==0:
                    v="0"+v
                else:
                    v="1"+v
            solutions.append(v)
        except:
            pass

print("solve all possible linear systems, where" )
print("")
print("H =")
print(A)
print("")
print("b =")
print(b)
print("")
print("solutions =")
print(solutions)

solve all possible linear systems, where

H =
[[1 1 0 0]
 [1 1 1 0]]

b =
[[1]
 [0]]

solutions =
['111010', '110110']


In [106]:

perm=[i for i in range(n)]
ancillas=[i for i in range(n,n+r)]

H=np.arange((n+1)*r).reshape(r,n+1)
c=n+r

for j in range(n+1):
    for i in range(r):
        H[i,j]=c
        c+=1


In [107]:
c=create_qibo_circuit(n,r)

#initialize matrix in quantum register
for i in range(r):
    for j in range(n):
        if A[i, j] == 1:
            c.add(gates.X(H[i, j]))
for i in range(r):
    if b[i] == 1:
        c.add(gates.X(H[i, n]))
        
#add pivots and create upper triangular matrix
for j in range(r-1):
    set_ancillas_to_num(c,ancillas,r)
    for i in range(n):
        if j<=i:
            add_negates_for_check(c,ancillas,r-j)
            add_pivot(c,j,i,H,n,r,ancillas+[perm[i]])
            lower_col_elimination(c,j,i,H,n,r,ancillas+[perm[i]])
            add_negates_for_check(c,ancillas,r-j)
        sub_one(c,ancillas,[perm[i]]) 

#perform backsubstitution
for j in range(r-1):
    set_ancillas_to_num(c,ancillas,r)
    for i in reversed(range(n)):
        if r-j-1<=i:
            add_negates_for_check(c,ancillas,r-j)
            upper_col_elimination(c,r-j-1,i,H,n,r,ancillas+[perm[i]])
            add_negates_for_check(c,ancillas,r-j)
        sub_one(c,ancillas,[perm[i]])

for i in range(r):
    add_one(c,ancillas[::-1],[H[i][n]])

#add measurements
c.add(gates.M(*([H[i,n] for i in range(r)]+perm+ancillas)))


In [108]:
ancillas[::-1]

[5, 4]

In [109]:
print(c.draw())

q0 : ─RY─o──────────────────────────o─o─o─o─o─o─o─o─o─o───o─o───────────────────────────────────────────────────────────────────────────────────────────o─o─────────M─
q1 : ────|─RY─o─RY─o────────────────|─|─|─|─|─|─|─|─|─|───|─|───o─o─o─o─o─o─o─o───o─o───────────────────────────────────────────────────────────o───o─o─|─|─────────M─
q2 : ────|─|──|─|──|─RY─o─X─o───────|─|─|─|─|─|─|─|─|─|───|─|───|─|─|─|─|─|─|─|───|─|───o─o─o─o─o─o───o─o─────────────────────────────o───o─o───|───|─|─|─|─────────M─
q3 : ────|─|──|─|──|─|──|─|─|─X─o───|─|─|─|─|─|─|─|─|─|───|─|───|─|─|─|─|─|─|─|───|─|───|─|─|─|─|─|───|─|───o─o─o─o───o─o───o───o─o───|───|─|───|───|─|─|─|─────────M─
q4 : ────x─o──X─|──x─o──X─|─x─o─X─X─o─o─o─o─o─o─o─o─o─o─X─X─o─X─o─o─o─o─o─o─o─o─X─X─o─X─o─o─o─o─o─o─X─X─o─X─o─o─o─o─X─X─o─X─o─X─X─o─X─o─X─X─o─X─o─X─X─o─X─o─X───X───M─
q5 : ─X──x──────o──x──────o─x─────X─o─o─o─o─o─o─o─o─o─o─────X───o─o─o─o─o─o─o─o─────X───o─o─o─o─o─o─────X───o─o─o─o─────X─X─o─────X───o─────X───o─────X───X─o─X─o─X─M

In [110]:

m=c(nshots=10000)
m.frequencies()
#<--+ r + --><----+ n +----><------+ r +------> | only log(r) needed for weight (will be updated soon)
# solution     permutation    weight(solution)  | weight given in binary

Counter({'01110001': 1600,
         '11001110': 1696,
         '11010110': 1651,
         '11011010': 1723,
         '11100110': 1677,
         '11101010': 1653})

In [111]:
solutions

['111010', '110110']

In [79]:
from math import ceil
def qubit_amount(n,r):
    return n+ceil(np.log2(r+1))+(n+1)*r

In [104]:
qubit_amount(n,r)

21